<a href="https://colab.research.google.com/github/PrincetonUniversity/python-code-cleanup/blob/main/python_cleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Code Cleanup: Tackle These 10 Common Issues

## Introduction

Python does a lot of things really well.  It's easy to read, fun to write, and is a great "glue" for performing system calls or interacting with performant, compiled code.  Open source and scientific libraries have secured a place for python in research code, allowing you to get more done in less time.


Python does a few things poorly.  Because python is not compiled, the computer isn't able to perform simple optimizations to speed up your code the way C++ or Java can.  Because python isn't strictly typed, it has to carry around a lot of extra information on what variables contain and what functions they can call.  This provides a lot of flexibility, but at the cost of speed.  When you append to a list in a for loop, there is overhead to look up the append function, not to mention having to grow the list occasionally.  In general, more lines of python code will be slower than fewer.


Here we will cover 10 common mistakes of new programmers.  Many result from learning a compiled language first and relying on c-style for loops.  Others are because some python-specific constructs are hard to grasp the first time you learn about them.  You'll learn why the original methods are slow or hard to read and what to replace them with.


Many of the replacements should be "drop in" replacements and you can perform them on any code base.  With any refactoring (code changing), it's always a good idea to have some *tests* in place and use *version control*.  Tests can be as simple as saving an output before changing things then comparing the output with your changed version.  Version control can just be copying your code somewhere as a backup.  Ideally you'd use a pytest suite to cover all your code and git, but we won't cover those here.  

In summary:
- python is slow
- correct code is better than easy to read code
- easy to read code is better than fast code

In [ ]:
# we will use this example for the first few exercises
import re
import random
import itertools

rng = random.Random(12345)

names = [
    "abby",
    "beth",
    "cathie",
    "deborah",
    "ellie",
    "francine",
    "grace",
    "helen",
    "irene",
    "jess",
    "kate",
]
statements = [
    "[10] is nice and round",
    "crazy [8]s",
    "8[6]7 5309",
    "[4] corners on a square",
    "I think that [2] is the best",
    "[0], how are your corner cases?",
    "When I look back" * 100 + "[11]",
    "[12] the number is here but your regex will read to the " + "end " * 100,
    "This number is far too large, but I like 100, start with [19]",
    "[13]",
    "[15]",
]

# going to repeat those a few times to make improvements more noticible
repeats = 100
names = [name + str(rng.randint(0, 10000)) for name
         in itertools.chain.from_iterable(itertools.repeat(names, repeats))]
statements = list(itertools.chain.from_iterable(itertools.repeat(statements, repeats)))


def get_fibs(names: list, statements: list):
  result = {}
  for name in names:
    index = names.index(name)
    statement = statements[index]
    value = parse_line(statement)
    result[name] = fibonacci(value)
  return result

def fibonacci(n: int) -> int:
  if n > 1:
    return fibonacci(n-1) + fibonacci(n-2)
  if n == 1:
    return 1
  return 0

def parse_line(line: str) -> int:
  match = re.match(".*\[(\d+)].*", line)
  return int(match.group(1))

%timeit get_fibs(names, statements)

## Cache your calls

I don't always use this, but the `fibonacci` function *really* benefits from caching.  Let's think about a call to `fibonacci(10)`.  The first round of recursion calls `fibonacci(9) + fibonacci(8)`, `fibonacci(9) = fibonacci(8) + fibonacci(7)` and you can already see we have to determine `fibonacci(8)` twice.  Going through the entire exercise, you get the following counts:

In [ ]:
def print_calls(n):
  calls = [0] * (n +1)
  def fibonacci(n: int) -> int:
    calls[n] += 1
    if n > 1:
      return fibonacci(n-1) + fibonacci(n-2)
    if n == 1:
      return 1
    return 0
  result = fibonacci(n)
  print(f"fibonacci({n}) = {result}")
  for i in range(n, 0, -1):
    print(f"fibonacci({i})\t{calls[i]}")
print_calls(10)

### Exercise

We already have a base case for `n == 1`, what would happen to your call counts if you added base cases for `n == 2`, `n == 3`, `n == 10`?

Clearly we can vastly speed up our function if we add in more base cases to short circuit the logic for the longer computation.  The downsides are more lines of code to write, maintain and ship.  The better solution is to store and reuse intermediate computations, a method called "caching".  In python, this is a simple function decorator.

In [ ]:
import functools

def print_calls(n):
  calls = [0] * (n +1)
  @functools.cache
  def fibonacci(n: int) -> int:
    calls[n] += 1
    if n > 1:
      return fibonacci(n-1) + fibonacci(n-2)
    if n == 1:
      return 1
    return 0
  result = fibonacci(n)
  print(f"fibonacci({n}) = {result}")
  for i in range(n, 0, -1):
    print(f"fibonacci({i})\t{calls[i]}")
print_calls(10)

Now once a value is calculated once, python remembers the return value and just returns that the next time.  It is effectively storing a dictionary of inputs and outputs and only running your function if the answer is not already stored.

Why not cache everything?
> Time/space tradeoffs.  This is common in computing, you can save time by storing the result, but you have to store the data somewhere.  Think about adding in more base cases to your recursion, it's faster but now you have more code to manage.

So when should you use caching?
- **You call your function a lot.**  Recursion is a good example since each top-level call has many recursive calls.
- **Your function has *a few* inputs.**  Limiting to say integers less than 1000 is reasonable to cache, but a function operating on strings or doubles has too many possible inputs.
- **Your function is a pure function.**  A method that depends on some object state or external value cannot be cached.
- **Your function is expensive.**  The time/space tradeoff is easier to satisfy if your function is computationally expensive.  While fibonacci calculations are just addition, the function calls and recursion are expensive and grow exponentially.


### Exercise
Add caching to our example code.  What is the new runtime?

In [ ]:
# we will use this example for the first few exercises
import re
import random
import itertools

rng = random.Random(12345)

names = [
    "abby",
    "beth",
    "cathie",
    "deborah",
    "ellie",
    "francine",
    "grace",
    "helen",
    "irene",
    "jess",
    "kate",
]
statements = [
    "[10] is nice and round",
    "crazy [8]s",
    "8[6]7 5309",
    "[4] corners on a square",
    "I think that [2] is the best",
    "[0], how are your corner cases?",
    "When I look back" * 100 + "[11]",
    "[12] the number is here but your regex will read to the " + "end " * 100,
    "This number is far too large, but I like 100, start with [19]",
    "[13]",
    "[15]",
]

# going to repeat those a few times to make improvements more noticible
repeats = 100
names = [name + str(rng.randint(0, 10000)) for name
         in itertools.chain.from_iterable(itertools.repeat(names, repeats))]
statements = list(itertools.chain.from_iterable(itertools.repeat(statements, repeats)))

def get_fibs(names: list, statements: list):
  result = {}
  for name in names:
    index = names.index(name)
    statement = statements[index]
    value = parse_line(statement)
    result[name] = fibonacci(value)
  return result

def fibonacci(n: int) -> int:
  if n > 1:
    return fibonacci(n-1) + fibonacci(n-2)
  if n == 1:
    return 1
  return 0

def parse_line(line: str) -> int:
  match = re.match(".*\[(\d+)].*", line)
  return int(match.group(1))

%timeit get_fibs(names, statements)

## Pythonic looping

Coming from C-style languages, for loops are mostly restricted to iterating over an index and using that to access array elements.  Python loops use iterators and generators in a way that is largely supported, performant, and fun.  If you have `for i in range(len(array)):` you aren't using python to its fullest!  

Get comfortable with directly accessing the values from nearly any collection, list, dict, set, tuples, or numpy arrays!  Since the iterator protocol is easy to extend, most libraries that implement a collection will also support access through a for loop.
```python
# BAD
array = list(range(10))  # some data
for i in range(len(array)):
  value = array[i]
  # do work

# BETTER!
array = list(range(10))  # some data
for value in array:
  # do work
```

There are several reasons direct iteration is superior.
1. It is clearer.  You want to do something with every element in a list, the syntax is nearly english.
2. It is performant.  You don't have to determine the length of your collection and create a range object.  More importantly, you don't have to then access the value from the collection.  Depending on the collection, that could be costly.

Expanding beyond direct iteration, you can enhance your for loops with functions like `sorted` or `reversed`.  With a dictionary, you can directly iterate over keys and values using the `.items()` function.  More on that later...

Our code example already uses `for name in names`, but we will evetually update that once we cover a few more changes.  For your own code, search for `range(len` and try to replace everyone you find!

### But I can't use direct iteration 😞

Well not with that attitude!  Most intro courses will leave you with a basic example, but in real applications you have to do more sophisticated programming than `# do work`.  But you can do things like keep track of your index or work with multiple collections (at once or sequentially) along with more advanced combinations, nested loops, and more.  Maybe the problem isn't with the loop, but your choice of data structures.  If indices are used for relating items between multiple lists, you probably want a dict or data table.

Don't be discouraged and fall back into using indices, think of it like a puzzle and keep experimenting.


## Find the itertools function for you

Itertools is a builtin library that provides most iteration functions you can think of.  Repeat an item forever, sure!  Group list items into batches?  Chain multiple lists, tee iterators, produce a cartesian product?  All and more!

Python also has builtin functions that you don't have to import.  Things like zip, all, any, filter, map, sorted, reversed, and enumerate.

Before you go and dive into complex chains of iterables, be aware that the most clever and performant solution is not always the best.  A verbose for loop that you can understand is probably better than 3 nested function calls you found on stack overflow.  But with that warning, let's work from fundamental to more complex.

### zip and enumerate for fun and profit

The first pattern that breaks a simple `for value in array` loop is when you have multiple collections that need to be addressed in step.  Say you have a list of subplots, x values, y values, and colors.  You may want to do something like
```python
for i in range(len(subplots)):
  subplot = subplots[i]
  x_val = x_values[i]
  y_val = y_values[i]
  color = colors[i]
  subplot.plot(x_val, y_val, color)
```

When you need to correlate items between collections, reach for zip.  Zip emits one item from each collection at each loop iterations.  When any collection is fully traversed, the loop exits.  If you want to instead zip until all collections are traversed, look at `itertools.zip_longest`, which allows you to specify a default value to pass for empty collections.  Using zip for our previous example:
```python
for subplot, x_val, y_val, color in zip(subplots, x_values, y_values, colors):
  subplot.plot(x_val, y_val, color)
```

Of course, sometimes you *need* the loop counter for displaying a label or setting a value in another list.  You can still stay away from `range(len` with `enumerate`.  Enumerate emits the loop counter and the item on each loop iteration.
```python
for i, value in enumerate(array):
  print(f"array[{i}] = {value}")
```
As a bonus, enumerate accepts the starting value, so you can get rid of your `i + 1` logic throughout the loop body.  You can nest other iterables into enumerate, just be careful in unpacking.  Adding a title to our subplot example:
```python
for i, (subplot, x_val, y_val, color) in enumerate(zip(subplots, x_values, y_values, colors), start=1):
  subplot.plot(x_val, y_val, color)
  subplot.set_title(f"Plot #{i}")
```

#### Exercise
Can you improve the `get_fibs` loop with enumerate?  How about zip?  Why do you think you get an improvement?

In [ ]:
# we will use this example for the first few exercises
import re
import random
import itertools
import functools

rng = random.Random(12345)

names = [
    "abby",
    "beth",
    "cathie",
    "deborah",
    "ellie",
    "francine",
    "grace",
    "helen",
    "irene",
    "jess",
    "kate",
]
statements = [
    "[10] is nice and round",
    "crazy [8]s",
    "8[6]7 5309",
    "[4] corners on a square",
    "I think that [2] is the best",
    "[0], how are your corner cases?",
    "When I look back" * 100 + "[11]",
    "[12] the number is here but your regex will read to the " + "end " * 100,
    "This number is far too large, but I like 100, start with [19]",
    "[13]",
    "[15]",
]

# going to repeat those a few times to make improvements more noticible
repeats = 100
names = [name + str(rng.randint(0, 10000)) for name
         in itertools.chain.from_iterable(itertools.repeat(names, repeats))]
statements = list(itertools.chain.from_iterable(itertools.repeat(statements, repeats)))

def get_fibs(names: list, statements: list):
  result = {}
  for name in names:
    index = names.index(name)
    statement = statements[index]
    value = parse_line(statement)
    result[name] = fibonacci(value)
  return result

@functools.cache
def fibonacci(n: int) -> int:
  if n > 1:
    return fibonacci(n-1) + fibonacci(n-2)
  if n == 1:
    return 1
  return 0

def parse_line(line: str) -> int:
  match = re.match(".*\[(\d+)].*", line)
  return int(match.group(1))

%timeit get_fibs(names, statements)

### Niche builtins

I don't always need these functions, but sometimes it's nice to not have to write your own:
- `any` and `all`.  If you have a list of `True` and `False` values, any and all can test if at least one of them (`any`) are true or if all of them (`all`) are true.  Nice for testing.
- `sorted`.  Return a list of the input sequence in sorted order.  As a bonus, you can choose what to sort by (say the second element of a tuple, an object attribute, etc).
- `reversed`.  Return a list in the opposite order.  Better than dealing with range nonsense!

#### Exercise
Above, I used the following for printing the number of calls for each input argument:
```python
  for i in range(n, 0, -1):
    print(f"fibonacci({i})\t{calls[i]}")
```
Can you propose an improvement?

### Builtins to (mostly) avoid
Map and filter allow you to alter a collection using function calls.  Say we have our list of 10 numbers:
```
data = list(range(10))
```
Map allows you to apply a function to each number in the list to produce a new list of the results.  Filter returns values in a list which pass some test, or predicate.  Let's see it in action:

In [ ]:
data = list(range(10))
print("Mapping x -> x^2")
print(list(map(lambda x: x ** 2, data)))
print("Filter values that are greater than 5")
print(list(filter(lambda x: x > 5, data)))

Map and filter can be more flexible and directly follow a functional paradigm, but for simple applications are much harder to read than list comprehensions.  More on those later.

### Itertools
The itertools module contains several useful functions and when combined can provide really complex loop constructs in a few lines.  The module is in the standard library but you do have to import it to use it.  [Documentation here](https://docs.python.org/3/library/itertools.html)

#### Exercises

Replace the following loops with equivalent itertools functions.

In [ ]:
import itertools

numbers = list(range(3))
letters = 'abcde'
symbols = 'αβγδ'

# nesting for loops
nests = []
for number in numbers:
  for letter in letters:
    for symbol in symbols:
      nests.append((number, letter, symbol))
print(f"{nests=}")

# flatten list
flattened = []
for l in (numbers, letters, symbols):
  flattened += l
print(f"{flattened=}")

# get indexes 2 to 6 of combined lists
slices = []
for i, number in enumerate(numbers):
  if 2 <= i < 6:
    slices.append(number)

for i, letter in enumerate(letters, i+1):
  if 2 <= i < 6:
    slices.append(letter)

for i, symbol in enumerate(symbols, i+1):
  if 2 <= i < 6:
    slices.append(symbol)
print(f"{slices=}")

# symbol * number
times = []
for number, symbol in zip(numbers, symbols):
  times.append(symbol * number)
print(f"{times=}")

# combine and batch of 4
batches = []
for i, number in enumerate(numbers):
  if i % 4 == 0:
    batches.append([])
  batches[-1].append(number)

for i, letter in enumerate(letters, i+1):
  if i % 4 == 0:
    batches.append([])
  batches[-1].append(letter)

for i, symbol in enumerate(symbols, i+1):
  if i % 4 == 0:
    batches.append([])
  batches[-1].append(symbol)
print(f"{batches=}")

#### Solutions

In [ ]:
import itertools
from collections import defaultdict

numbers = list(range(3))
letters = 'abcde'
symbols = 'αβγδ'

# converting to lists to test for equality, usually want iterators
lists = (numbers, letters, symbols)

# nesting for loops
inests = list(itertools.product(*lists))
assert nests == inests

# flatten list
iflattened = list(itertools.chain(*lists))
assert flattened == iflattened

# get indexes 2 to 6 of combined lists
islices = list(itertools.islice(itertools.chain(*lists), 2, 6))
assert slices == islices

# symbol * number
itimes = list(itertools.starmap(lambda x, y: x * y, zip(numbers, symbols)))
assert times == itimes

# combine and batch of 4, python >=3.12
# ibatches = list(itertools.batched(itertools.chain(*lists), n=4))
# assert batches == ibatches

ibatches = list(
    list(val[1] for val in g)  # get just the values
    for k, g in itertools.groupby(enumerate(itertools.chain(*lists)), lambda x: x[0] // 4)  # group into batches
)
assert batches == ibatches

## Using the right data structure

Many new python developers over-use the builtin list data structure.  Lists are easy to grasp coming from languages with arrays as the standard data structure while sets, dicts and tuples seem a bit more exotic or harder to use.  However lists are usually the least performant, especially when you use them to emulate features you get for free in the other data structures.  Use lists if you want to change (add or remove) items and the values don't need to be unique.

### Sets: mutable and unique

Consider the following to add items into a list, only if the value is not already in the list:

In [ ]:
def unique_list(numbers):
  my_list = list(range(numbers))
  new_list = list(range(numbers*2))
  for value in new_list:
    if value not in my_list:
      my_list.append(value)
%timeit unique_list(10_000)

This works, but is slow because you have to search for each incoming value (`value not in my_list`) and then you append the value.  Sets have the invariant that all items are unique.  Membership is tested using a fast hashing method, instead of iterating over the entire list.  When you have unique items, you almost always want to use a set.  The only exception is if order of the items is important.

In [ ]:
def unique_set(numbers):
  my_set = set(range(numbers))
  new_set = set(range(numbers*2))
  for value in new_set:
    if value not in my_set:
      my_set.add(value)
%timeit unique_set(10_000)

Sets also have methods for performing mathematical set operations like intersection (`&`) and union (`|`).  These are faster than the equivalent for loops because you bypass the `add` method lookup and execution.  Here we want the union:

In [ ]:
def unique_set(numbers):
  my_set = set(range(numbers))
  new_set = set(range(numbers*2))
  my_set = my_set | new_set
%timeit unique_set(10_000)

### Tuples: immutable lists

Tuples aren't used enough in python.  Lists are an easy default but they have performance implications even in getting an item.  Tuples are like lists in that they allow duplicates but you cannot add or remove items.  When you see a list in your code, consider if you actually change it during execution.  One common example is in for loops:
```python
for color in ['red', 'green', 'blue']:
  ...
# could be
for color in ('red', 'green', 'blue'):
  ...
```
This small example probably won't matter, but it's good practice to default to tuples and only use lists if you decide you have to add or remove items.

In [ ]:
def list_get(numbers):
  my_list = list(range(numbers))
  for i in range(numbers):
    n = my_list[i]

%timeit list_get(100_000)

In [ ]:
def tuple_get(numbers):
  my_tuple = tuple(range(numbers))
  for i in range(numbers):
    n = my_tuple[i]

%timeit tuple_get(100_000)

### Dicts: relating two collections

One of the least performant and hardest to read anti-patterns is using two lists instead of a dict.  Something like:
```python
names = ['steve', 'jim', 'howard']
foods = ['apple', 'banana', 'citrus']
```
where the relation between names and foods is solely through indexing, e.g. `names[0]` corresponds to `foods[0]`.  This is hard to read and reason about because the relation is not enforced or clear.  How do you know *those* two lists are related?  It's not performant because of issues related to lists.

Dicts relate keys, like an index, to values which can be any object.  The underlying key structure is similar to a set, so you get fast checks for membership and lookups.  keys and values are both iterables and can be used directly in a loop, don't cast to lists!



In [ ]:
# bad
names = ['steve', 'jim', 'howard']
foods = ['apple', 'banana', 'citrus']

for name, food in zip(names, foods):
  print(f"{name} likes {food}")

# direct initialization
favorite_foods = {
    'steve': 'apple',
    'jim': 'banana',
    'howard': 'citrus'
}

# from lists
favorite_foods = dict(zip(names, foods))

for name in list(favorite_foods.keys()):  # boo, don't need a list
  food = favorite_foods[name]  # yuck
  print(f"{name} likes {food}")

for name in favorite_foods:  # better, default iterator is over keys
  food = favorite_foods[name]  # yuck
  print(f"{name} likes {food}")

for name, food in favorite_foods.items():  # best
  print(f"{name} likes {food}")


#### Exercise

Replace the two lists of names and statements with a dict.

In [ ]:
# we will use this example for the first few exercises
import re
import random
import itertools
import functools

rng = random.Random(12345)

names = [
    "abby",
    "beth",
    "cathie",
    "deborah",
    "ellie",
    "francine",
    "grace",
    "helen",
    "irene",
    "jess",
    "kate",
]
statements = [
    "[10] is nice and round",
    "crazy [8]s",
    "8[6]7 5309",
    "[4] corners on a square",
    "I think that [2] is the best",
    "[0], how are your corner cases?",
    "When I look back" * 100 + "[11]",
    "[12] the number is here but your regex will read to the " + "end " * 100,
    "This number is far too large, but I like 100, start with [19]",
    "[13]",
    "[15]",
]

# going to repeat those a few times to make improvements more noticible
repeats = 100
names = [name + str(rng.randint(0, 10000)) for name
         in itertools.chain.from_iterable(itertools.repeat(names, repeats))]
statements = list(itertools.chain.from_iterable(itertools.repeat(statements, repeats)))

def get_fibs(names: list, statements: list):
  result = {}
  for index, name in enumerate(names):
    statement = statements[index]
    value = parse_line(statement)
    result[name] = fibonacci(value)
  return result

@functools.cache
def fibonacci(n: int) -> int:
  if n > 1:
    return fibonacci(n-1) + fibonacci(n-2)
  if n == 1:
    return 1
  return 0

def parse_line(line: str) -> int:
  match = re.match(".*\[(\d+)].*", line)
  return int(match.group(1))

%timeit get_fibs(names, statements)

### Just use numpy

numpy is the de-facto numerical library for python.  It is space efficient because numpy arrays hold only a single type, say integer or floating point, which cuts down significantly on the type information python has to carry around with lists.  It is fast because under the hood operations call highly-optimized, compiled functions.  Common mathematical operations have dedicated functions so you don't have to write your own.  

Once you get creative with matrix operations, you can remove most of the loops from your code and replace them with numpy functions.  Note this can sometimes obfuscate your code so keep the clearer, original version in your unit tests to ensure outputs are equal and act as documentation.

If you do any sort of numerical operations on many items, spend time learning numpy.  We don't have the time here.

## Replace loops with comprehensions

New python developers shy away from list comprehensions because the syntax can be confusing, but they are succinct and actually faster than equivalent for loops.  Because of this, comprehensions are common in python code.

Starting with lists, let's make a list of sequential digits:

In [ ]:
def loop_list(number):
  result = []
  for i in range(number):
    result.append(number)
  return result

def list_comp(number):
  return [i for i in range(number)]

def list_func(number):
  return list(range(number))

%timeit loop_list(100_000)
%timeit list_comp(100_000)
%timeit list_func(100_000)

You can see comprehensions are about twice as fast as a loop.  Mostly this is due to avoiding calls to `append`.  Function calls in python are somewhat expensive and without compiling the computer isn't able to inline or optimize calls in loops.  The call to the `list` function is about twice as fast again, but it is less flexible.  List comprehensions make it easy to filter and nest loops:

In [ ]:
result = []
for fruit in ('apple', 'banana'):
  for i in range(5):
    if i % 2 == 0:
      result.append(f'{fruit} -> {i}')

print(result)

print([
    f'{fruit} -> {i}'
    for fruit in ('apple', 'banana')
    for i in range(5)
    if i % 2 == 0
])

You can use almost the same syntax for set and dictionary comprehensions.  Finally, if you use parentheses you create a generator, which is iterable but evaluated lazily

In [ ]:
print('a list', [i for i in range(5)])
print('a set', {i for i in range(5)})
print('a generator', (i for i in range(5)))
print('a dict', {i: i**2 for i in range(5)})

### Exercise

Replace the `dict` function below with a generator expression.  Replace the get_fibs implementation with a comprehension.

In [ ]:
# we will use this example for the first few exercises
import re
import random
import itertools
import functools

rng = random.Random(12345)

names = [
    "abby",
    "beth",
    "cathie",
    "deborah",
    "ellie",
    "francine",
    "grace",
    "helen",
    "irene",
    "jess",
    "kate",
]
statements = [
    "[10] is nice and round",
    "crazy [8]s",
    "8[6]7 5309",
    "[4] corners on a square",
    "I think that [2] is the best",
    "[0], how are your corner cases?",
    "When I look back" * 100 + "[11]",
    "[12] the number is here but your regex will read to the " + "end " * 100,
    "This number is far too large, but I like 100, start with [19]",
    "[13]",
    "[15]",
]

# going to repeat those a few times to make improvements more noticible
repeats = 100
data = dict(zip(
    (name + str(rng.randint(0, 10000)) for name in itertools.chain.from_iterable(itertools.repeat(names, repeats))),
    itertools.chain.from_iterable(itertools.repeat(statements, repeats))
    ))

def get_fibs(data: dict):
  result = {}
  for name, statement in data.items():
    value = parse_line(statement)
    result[name] = fibonacci(value)
  return result

@functools.cache
def fibonacci(n: int) -> int:
  if n > 1:
    return fibonacci(n-1) + fibonacci(n-2)
  if n == 1:
    return 1
  return 0

def parse_line(line: str) -> int:
  match = re.match(".*\[(\d+)].*", line)
  return int(match.group(1))

%timeit get_fibs(data)

## Efficient string building

There are two flavors of string building, combining different variables into a template and joining items from a collection.  Let's start with our favorite food example:

In [ ]:
favorite_foods = {
    'steve': ['apple', 'cheese', 'bread'],
    'jim': ['banana', 'peanut butter'],
    'howard': ['citrus', 'fish', 'butter'],
}

result = ''
for person, foods in favorite_foods.items():
  result += person + ' really likes '
  for i, food in enumerate(foods):
    result += food
    if i != len(foods) -1:
      result += ", "
  result += '!\n'
print(result)

### Exercise
In the code above, where are you combining into a template and where are you joining?  Can you explain the if statement?

### Formatting and f-strings

String building with `+` and `+=` is slow for the same reasons appending to a list is slow.  You have overhead for function calls and you have to resize your collection.  The first method of removing `+` is using formatting.  You format a string with a call to `format`.  f-strings use similar templating but you don't even need the function call:

In [ ]:
name = 'steve'
food = 'apples'

print(name + ' really likes ' + food + '!')  # slow, hard to read and type
print("{name} really likes {food}!".format(name=name, food=food))  # better, can pass template as argument
print(f"{name} really likes {food}!")  # good if you don't need to pass the string
    # ^ note this f!

x = 1
print('x = ' + str(x))  # can only add strings, have to force string here!
print('x = {x}'.format(x=x))  # format and f strings handle that for you
print(f'x = {x}')

Both format and f strings are fast.  f strings also have some nice formatting features that are handy to know.

In [ ]:
pi = 3.1415
alphabet = 'abcdefghijklmnopqrstuvwxyz'
number = 42

print(f'A variable with an equal prints the variable name {number=}')
print(f'You can even use expressions with it like {pi+number=}')
print(f'Most python code is allowed, like {len(alphabet)=} or {1+3=}')
print(f'Formatting floating numbers is handy but hard to remember {pi=:0.2f} vs {pi=}')
print(f'Or pad |{number:<40}|')
print(f'Or pad |{number:>40}|')
print(f'Use double {{braces}} to print braces {number}')

### Joining strings
The other common string building operation is joining tokens with some delimiter.  Say you have a list of items and you want to print them comma separated or print items on separate lines.  This can be difficult if you want to properly handle cases like a single item, no items, and you want to not have an "extra" separator at the beginning or end.

In [ ]:
def my_join(strings: list):
  result = ''
  for string in strings:
    result += string + ", "
  return result

print('empty list')
print(my_join([]))
print('one item')
print(my_join(['apple']))
print('more items')
print(my_join(['apple', 'banana']))

The builtin for the job is `join`.  The syntax is a little odd at first.  `join` is a string method on the *delimiter* and takes as arguments the string iterable you want joined.  Here are some examples.

In [ ]:
print(','.join([]))
print(', '.join(['apple']))
print(', '.join(['apple', 'banana']))
print(', '.join(str(i) for i in range(4)))  # can take generators, must be strings
print('|'.join('-'*5 for i in range(4)))  # change delimiter

And a speed comparison

In [ ]:
def py_join(strings: list):
  return ', '.join(strings)

strings = ['abcdefghijklmnopqrstuvwxyz'[i%26] for i in range(10_000)]
%timeit my_join(strings)
%timeit py_join(strings)

### Exercise

Use join and formatting to improve the following code:

In [ ]:
favorite_foods = {
    'steve': ['apple', 'cheese', 'bread'],
    'jim': ['banana', 'peanut butter'],
    'howard': ['citrus', 'fish', 'butter'],
}

result = ''
for person, foods in favorite_foods.items():
  result += person + ' really likes '
  for i, food in enumerate(foods):
    result += food
    if i != len(foods) -1:
      result += ", "
  result += '!\n'
print(result)

### Solution

In [ ]:
favorite_foods = {
    'steve': ['apple', 'cheese', 'bread'],
    'jim': ['banana', 'peanut butter'],
    'howard': ['citrus', 'fish', 'butter'],
}
print('\n'.join(
    f"{person} really likes {', '.join(foods)}!"
    for person, foods in favorite_foods.items()
))

## Supercharging regular expressions

Regular expressions are as powerful as they are terse.  Whole books have been written on regex's so we can't cover everything.  I just want to impart two points:
- regex's are evaluated greedily and stupidly
- the more specific, the better
- specifically for python, compile any regexes you use more than once.

Let's focus on our regex from earlier:
```python
line = "I think that [2] is the best"
re.match(".*\[(\d+)].*", line)
```

Here `match` must match at the start of the line.  The first token `.*` matches to anything.  As such, the first step in the evaluation is to match the entire line.  Next, the engine sees the last character `t` doesn't match a literal `[` so it has to backtrack.  If your string is long and the opening bracket in the beginning, this can be a bit expensive.

### Exercise
Can you think of a better matching statement than `.*` for the first part of the string?

We don't really need the entire string, so why do we bother matching against it?  A better option would be to just extract the portion we need, a number in braces.  The function for the job is `search`.  Then our regex can simply be `r"\[(\d+)]"`.  By being more specific we can make our matching robust and fast.

The final way to supercharge regular expressions in python is to pre-compile them.  From the documentation:
```python
#The sequence

prog = re.compile(pattern)
result = prog.match(string)
#is equivalent to

result = re.match(pattern, string)
```
It also states that
> The compiled versions of the most recent patterns passed to re.compile() and the module-level matching functions are cached, so programs that use only a few regular expressions at a time needn't worry about compiling regular expressions.

I would still compile if regular expressions are a large part of your project.  You may only use one, but maybe libraries you import use enough to fill the cache.

Here are some timing experiments:

In [ ]:
import re

line = "[12] the number is here but your regex will read to the " + "end " * 10_000

def match_parse(line):
  match = re.match(".*\[(\d+)].*", line)
  return int(match.group(1))

def search_parse(line):
  match = re.search("\[(\d+)]", line)
  return int(match.group(1))

REGEX = re.compile("\[(\d+)]")
def compiled_parse(line):
  match = REGEX.search(line)
  return int(match.group(1))

%timeit match_parse(line)
%timeit search_parse(line)
%timeit compiled_parse(line)

## Using context managers

Context managers are another great dose of python syntactic sugar.  They allow you to ensure a function is called at the start and end of a code block.  Uses can be as diverse as locking a semaphore, connecting to a database, or temporarily setting numeric precision.  Here we will focus on file managers.  Coming from other languages you are probably familiar with the following:
```python
my_file = open('file.txt')
# do work
my_file.close()
```
The problems with explicitly calling close are that you may forget to call it and if your work raises an exception, it may never be called.  Context managers solve both these issues:
```python
with open('file.txt') as my_file:
  # do work
```
If you have lots of files, consider using an [`ExitStack`](https://docs.python.org/3/library/contextlib.html#contextlib.ExitStack).  For custom context managers, look into the [contextmanager decorator](https://docs.python.org/3/library/contextlib.html#contextlib.contextmanager).

## Clean up your print statements

We all have used print statements for debugging code, but did you know those can also have performance implications?  Writing and synchronizing outputs to the terminal can really slow down your code.  Make sure you search and remove any unnecessary print statements.  Look into logging if you want a tunable handle on your program state.  Automated solutions with pre-commit integration are also available.

## Import what you need

A final quick tip is to only import what you need.  This works two ways: avoiding star imports and only importing libraries you use in a file.

```python
from pandas import *
my_df = DataFrame()
# vs
import pandas
my_df = pandas.DataFrame()
```
The first may look like it saves typing, but it pollutes your name space, makes it harder to see which function is called and from where, and can slow down execution by importing every function in a library.

```
import torch
import pandas
import numpy

print('hello world')
```
This seems silly, but often during refactoring you will pull functions into other files and leave vistigial imports.  These unused imports can add to execution slowdowns as you import (and compile) libraries unnecessarily.  Again there are automated options with pre-commit.

## Closing
We choose python for different reasons, it is easy to read, fun to write, and a great starting language.  Using python-specific constructs helps you write more pythonic code that is easier for other developers to understand.  Once you are familiar with list comprehensions, it's easier to follow a one-line comprehension than the equivalent for loop.  You may be surprised that pythonic code also frequently runs faster.  This is because the language can use shortcuts for performing operations; e.g. bypassing `list.append` in list comprehensions.

I hope you've learned at least one trick to make your code fast, pythonic, and fun!